# CDNow消费行为数据分析-task10

首先看下这个数据集，4个特征：ID，购买日期，订单数，订单金额

首先导入基本的库

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime

这四个字段处理啊上有个好处，都是数字，没有字符，感觉分析上可能也方便些

特征head定义：ID，Date，Count，Price

In [ ]:
df = pd.read_table('CDNOW.txt',names = ['ID','Date','Count','Price'],sep = '\s+')

In [ ]:
df.head()

In [ ]:
df.describe()

通过对数据基本的分析，可以看出OrderPirce的最大值是1286.01，最小值是0，平均值是36，可以有个基本认识，价格高的商品不是很多，因为平均价格时36左右，那是不是要考虑下1286.01的商品价格有没有可能数据有问题

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df['Date'] = pd.to_datetime(df.Date, format='%Y%m%d')

In [ ]:
df['Month'] = df.Date.values.astype('datetime64[M]')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

依然没空值，好开心

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
plt.figure(figsize=(20,15))

In [ ]:
plt.subplot(221)
df.groupby('Month')['Price'].sum().plot(fontsize=24) 
plt.title('TotalPrice',fontsize=24) 

In [ ]:
plt.subplot(222)
df.groupby('Month')['Date'].count().plot(fontsize=24)
plt.title('PayNum',fontsize=24) 

In [ ]:
plt.subplot(223)
df.groupby('Month')['Count'].sum().plot(fontsize=24)
plt.title('TotalPrice',fontsize=24)

In [ ]:
plt.subplot(224)
df.groupby('Month')['ID'].apply(lambda x:len(x.unique())).plot(fontsize=24)
plt.title('PersonNum',fontsize=24)
plt.tight_layout()
plt.show()

In [ ]:
person = df.groupby('ID').sum()

In [ ]:
person.describe()

In [ ]:
person.query('Price < 500').plot.scatter(x='Price',y='Count')

In [ ]:
person.Price.plot.hist(20)

In [ ]:
person.query("Price< 800")['Price'].plot.hist(bins=20)

In [ ]:
person.query('Count < 100').Count.hist(bins = 40)

In [ ]:
order_diff = df.groupby('ID').apply(lambda x:x['Date'] - x['Date'].shift())


In [ ]:
order_diff.head(10)

In [ ]:
order_diff.describe()

In [ ]:
plt.figure(figsize=(15,5))
plt.hist((order_diff / np.timedelta64(1, 'D')).dropna(), bins = 50)
plt.xlabel('消费周期',fontsize=24)
plt.ylabel('频数',fontsize=24)
plt.title('用户消费周期分布图',fontsize=24);

In [ ]:
orderdt_min=df.groupby('ID').Date.min()
orderdt_max=df.groupby('ID').Date.max()
(orderdt_max-orderdt_min).head()

In [ ]:
(orderdt_max-orderdt_min).mean()

In [ ]:
((orderdt_max-orderdt_min)/np.timedelta64(1,'D')).hist(bins=15)

In [ ]:
life_time = (orderdt_max - orderdt_min).reset_index()

In [ ]:
life_time.head()

In [ ]:
plt.figure(figsize=(10,5))
life_time['life_time'] = life_time.Date / np.timedelta64(1,'D')
life_time[life_time.life_time > 0].life_time.hist(bins = 100, figsize = (12,6))

In [ ]:
life_time[life_time.life_time>0].Date.mean()

In [ ]:
rfm = df.pivot_table(index='ID',values=['Price','Date','Count'],aggfunc={'Price':'sum','Date':max,'Count':'sum'})

In [ ]:
rfm.head()

In [ ]:
rfm['R'] = (rfm['Date'].max() - rfm['Date']) / np.timedelta64(1,'D')

In [ ]:
rfm.rename(columns={'Price':'M','Count':'F'},inplace=True)

In [ ]:
def get_rfm(x):
    level = x.apply(lambda x:'1' if x >=0 else '0')
    label = level['R'] + level['F'] + level['M']
    d = {'111':'价值客户','011':'保持客户','101':'挽留客户','001':'发展客户','110':'一般价值客户','010':'一般保持客户','100':'一般挽留客户','000':'一般发展客户'}
    result = d[label]
    return result

In [ ]:

rfm['label'] = rfm[['R','F','M']].apply(lambda x:(x-x.mean()) / x.std()).apply(get_rfm,axis=1)

In [ ]:
rfm.head()

In [ ]:
rfm.groupby('label').sum()

In [ ]:
rfm.groupby('label').size()

In [ ]:
df1 = df.pivot_table(index = "ID",
                     columns = "Month",
                     values = 'Date',
                     aggfunc = 'count').fillna(0)
df1.head() 

In [ ]:
df2 = df1.applymap(lambda x:1 if x>0 else 0)

In [ ]:
df2.tail()

In [ ]:
def active_status(data):
    status=[]
    for i in range(18): #共18个月
        
        #若本月没有消费
        if data[i]==0:
            if len(status)>0:#前面某月消费过，是老客
                if status[i-1]=='unreg':#前一个月不是首次消费，不是新客
                    status.append('unreg')#则本月也不是新客
                else:
                    status.append('unactive')#前一个月是首次消费，属于新客，则本月为不活跃用户
            else:
                status.append('unreg')#前面某月没有消费过，则本月也不是新客
         #若本月消费
        else:
            if len(status)==0:#前面没有消费过
                status.append('new')#则为新客
            else:#前面消费过
                if status[i-1] =='unactive':#前一个月没有消费，是不活跃用户
                    status.append('return')#本月为回流用户
                elif status[i-1]=='unreg':#前一个月没有消费，不是新客
                    status.append('new')
                else:#前一个月是首次消费
                    status.append('active')#本月为活跃用户
                    
    return status

In [ ]:
indexs=df['Month'].sort_values().astype('str').unique()
df3=df2.apply(lambda x:pd.Series(active_status(x),index=indexs),axis=1)
df3.head()

In [ ]:
df4=df3.replace('unreg',np.NaN).apply(lambda x:pd.value_counts(x)).fillna(0).T
df4

In [ ]:
u =df4.reset_index()
labels = u[['active','new','return','unactive']].columns
plt.figure(figsize=(15,5))
plt.stackplot(u['index'].astype(str).apply(lambda x:x[:-3]), u['active'],u['new'],u['return'],u['unactive'], labels=labels)
plt.xlabel('月份')
plt.ylabel('消费人数')
plt.title('每月的消费人数')
plt.legend(loc = 'upper left');

In [ ]:
df5=df4.apply(lambda x:x/x.sum(),axis=1)#每一层用户占总用户的比例
df5

In [ ]:
a = df.groupby('ID')['Date'].agg(['min','max']).reset_index()
new_old = (a['min'] == a['max']).value_counts().values

plt.pie(x = new_old,
       autopct = '%.1f%%',
       shadow = True,
       explode = [0.08,0],
       textprops = {'fontsize' : 11})
plt.axis('equal') 
plt.legend(['仅消费一次','多次消费'])

In [ ]:
pivoted_df=df.pivot_table(index='ID',columns='Month',values='Date',
                       aggfunc='count').fillna(0)
pivoted_df.head()
pivoted_df_transf=pivoted_df.applymap(lambda x: 1 if x>1 else np.nan if x==0 else 0)
pivoted_df_transf.head()

In [ ]:
df_duplicate =pd.DataFrame(pivoted_df_transf.sum()/pivoted_df_transf.count()).reset_index()
df_duplicate.columns = ['Date', 'DuplicatedRate']
df_duplicate['Date'] = df_duplicate.Date.astype(str).apply(lambda x:x[:-3])

plt.figure(figsize = (15,6))
plt.plot(df_duplicate.Date, df_duplicate.DuplicatedRate)
plt.xlabel('time', fontsize=24)
plt.ylabel('rebuy',fontsize=24)
# plt.ylim(0,1)
plt.title('change',fontsize=24)

In [ ]:

df_purchase = pivoted_df.applymap(lambda x:1 if x>1 else np.nan if x==0 else 0)

In [ ]:
df_purchase.apply(lambda x:x.sum()/x.count()).plot.bar()

In [ ]:
df_purchase_back = df_purchase.applymap(lambda x:1 if x>0 else np.nan)
df_purchase_back.head()

In [ ]:
def purchase_back(data):
    lenth = len(data)
    state = []
    for i in range(0,lenth-1):
        if data[i] == 1:
            if data[i+1]==1:
                state.append(1) #若本月已消费，下个月也消费置1
            else:
                state.append(0) #本月已消费，下月未消费置0
        else:
            state.append(np.nan) #本月未消费置np.nan
    state.append(np.nan)
    return state

In [ ]:
df_purchase_b = df_purchase_back.apply(purchase_back,axis=1)
df_purchase_b.apply(lambda x:x.sum()/x.count()).plot.bar()